In [4]:
#二維資料(2 levels)
import numpy as np
import pandas as pd

df1 = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
                  'key2':['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5), #randn(): return a sample(or samples) from the "standard normal" distribution
                  'data2':np.random.randn(5)})
df1

,key1,key2,data1,data2
0,a,one,0.507336,1.159769
1,a,two,-1.753581,-0.862490
2,b,one,-0.222361,-1.825556
3,b,two,-0.041857,2.146513
4,a,one,-0.353463,-0.966502


In [6]:
df1 = df1.set_index(keys = ['key1','key2'])
df1

data1     data2
key1 key2                    
a    one   0.507336  1.159769
     two  -1.753581 -0.862490
b    one  -0.222361 -1.825556
     two  -0.041857  2.146513
a    one  -0.353463 -0.966502

In [7]:
df1.columns.name = 'data'
df1

data          data1     data2
key1 key2                    
a    one   0.507336  1.159769
     two  -1.753581 -0.862490
b    one  -0.222361 -1.825556
     two  -0.041857  2.146513
a    one  -0.353463 -0.966502

In [9]:
df2 = df1.groupby(level = ["key1","key2"]).mean()
display(df2)
display(df2.unstack(level = "key1"))

data          data1     data2
key1 key2                    
a    one   0.076937  0.096633
     two  -1.753581 -0.862490
b    one  -0.222361 -1.825556
     two  -0.041857  2.146513

data     data1               data2          
key1         a         b         a         b
key2                                        
one   0.076937 -0.222361  0.096633 -1.825556
two  -1.753581 -0.041857 -0.862490  2.146513